In [8]:
import os
from git import Repo
from minio import Minio


In [15]:

# Ruta local donde se almacenará el repositorio
repo_dir = '../ETL-P/'

# Carpeta específica dentro del repositorio de la cual quieres tomar los archivos
target_folder = 'Data/'

def clone_or_update_repo():
    # Si el repositorio ya existe, lo actualiza
    if os.path.exists(repo_dir):
        repo = Repo(repo_dir)
        origin = repo.remotes.origin
        origin.pull()  # Actualiza el repositorio
    else:
        # Si no existe, clona el repositorio
        Repo.clone_from("https://github.com/David-I-X/ETL-P.git", repo_dir)

    # Crear una lista de archivos solo dentro de la carpeta especificada
    file_paths = []
    target_folder_path = os.path.join(repo_dir, target_folder)

    # Verifica si la carpeta existe
    if os.path.exists(target_folder_path):
        for root, dirs, files in os.walk(target_folder_path):
            for file in files:
                file_paths.append(os.path.join(root, file))
    else:
        print(f"La carpeta {target_folder} no existe en el repositorio.")

    return file_paths


In [16]:
clone_or_update_repo()

['../ETL-P/Data/air_quality_cleaned.parquet',
 '../ETL-P/Data/Calidad_aireL.parquet',
 '../ETL-P/Data/car_resale_prices_clean.csv',
 '../ETL-P/Data/cleaned_fhv_tripdata_2024-01.parquet',
 '../ETL-P/Data/cleaned_green_tripdata_2024-01.parquet',
 '../ETL-P/Data/cleaned_yellow_tripdata_part_1.parquet',
 '../ETL-P/Data/cleaned_yellow_tripdata_part_2.parquet',
 '../ETL-P/Data/cleaned_yellow_tripdata_part_3.parquet',
 '../ETL-P/Data/cleaned_yellow_tripdata_part_4.parquet',
 '../ETL-P/Data/cleaned_yellow_tripdata_part_5.parquet',
 '../ETL-P/Data/costo_operacional_vehiculos_clean.csv',
 '../ETL-P/Data/datos_geograficos_limpios.parquet',
 '../ETL-P/Data/datos_meteorologicos_limpios.parquet',
 '../ETL-P/Data/df_eafcs.parquet',
 '../ETL-P/Data/Df_vfed.parquet',
 '../ETL-P/Data/energy_clean.csv',
 '../ETL-P/Data/fhv_tripdata.parquet',
 '../ETL-P/Data/green_tripdata.parquet',
 '../ETL-P/Data/vehicle_counts_cleaned copy.parquet',
 '../ETL-P/Data/vehicle_counts_cleaned.parquet',
 '../ETL-P/Data/yello

In [17]:

# Conectar con MinIO
client = Minio(
    "redarcope.ddns.net:9000",  # Cambia a la IP de tu servidor MinIO si es remoto
    access_key="andres",     # Reemplaza con tu clave de acceso
    secret_key="andresarcope01",  # Reemplaza con tu clave secreta
    secure=False  # Cambia a True si usas HTTPS
)

# Ruta para almacenar archivos ya subidos
uploaded_files_log = '../ETL-P/Data/uploaded_files.txt/'

def get_uploaded_files():
    # Si el archivo no existe, crea uno vacío
    if not os.path.exists(uploaded_files_log):
        open(uploaded_files_log, 'w').close()

    # Lee los nombres de los archivos que ya han sido subidos
    with open(uploaded_files_log, 'r') as f:
        return set(f.read().splitlines())

def log_uploaded_file(file_name):
    # Añadir el nombre del archivo al registro de archivos subidos
    with open(uploaded_files_log, 'a') as f:
        f.write(file_name + '\n')

def upload_new_files_to_minio(file_paths):
    uploaded_files = get_uploaded_files()
    bucket_name = "data-inicio"

    # Crear bucket si no existe
    if not client.bucket_exists(bucket_name):
        client.make_bucket(bucket_name)

    # Subir solo archivos que no están en el registro
    for file_path in file_paths:
        file_name = os.path.basename(file_path)
        if file_name not in uploaded_files:
            client.fput_object(
                bucket_name,
                file_name,
                file_path
            )
            log_uploaded_file(file_name)  # Registrar el archivo subido
            print(f"Archivo {file_name} subido correctamente.")
        else:
            print(f"Archivo {file_name} ya existe en MinIO, omitiendo.")


In [ ]:
from airflow import DAG
from airflow.operators import PythonOperator
from datetime import datetime, timedelta

# Definir los argumentos por defecto para el DAG
default_args = {
    'owner': 'airflow',
    'depends_on_past': False,
    'start_date': datetime(2024, 9, 17),  # Cambia esta fecha según tu contexto
    'email_on_failure': False,
    'email_on_retry': False,
    'retries': 1,
    'retry_delay': timedelta(minutes=5),
}

# Crear el DAG
dag = DAG(
    'github_to_minio_etl_dag',
    default_args=default_args,
    description='Pipeline que extrae archivos de GitHub y sube a MinIO',
    schedule_interval=timedelta(days=1),  # Corre diariamente
)

# Función para clonar o actualizar el repositorio y obtener los archivos
def extract_from_github(**kwargs):
    return clone_or_update_repo()

# Función para cargar archivos nuevos a MinIO
def load_to_minio(**kwargs):
    ti = kwargs['ti']  # Obtén el XCom (datos pasados entre tareas)
    file_paths = ti.xcom_pull(task_ids='extract_from_github')
    upload_new_files_to_minio(file_paths)

# Definir las tareas del DAG
extract_task = PythonOperator(
    task_id='extract_from_github',
    python_callable=extract_from_github,
    dag=dag,
)

load_task = PythonOperator(
    task_id='load_to_minio',
    python_callable=load_to_minio,
    provide_context=True,  # Permite usar XCom
    dag=dag,
)

# Definir el orden de ejecución
extract_task >> load_task